# 🏥 Hands-On: Medical LLM Fine-Tuning Practice

## 📋 Table of Contents
1. [Environment Setup and Installation](#practice-1-environment-setup-and-installation)
2. [Loading Pre-trained Models](#practice-2-loading-pre-trained-models)
3. [Preparing Medical Datasets](#practice-3-preparing-medical-datasets)
4. [Configuring LoRA for PEFT](#practice-4-configuring-lora-for-peft)
5. [Training with Medical Instructions](#practice-5-training-with-medical-instructions)
6. [Model Evaluation and Testing](#practice-6-model-evaluation-and-testing)
7. [Inference and Deployment](#practice-7-inference-and-deployment)
8. [Monitoring and Optimization](#practice-8-monitoring-and-optimization)

---

## 🎯 Learning Objectives
- Implement LoRA-based parameter-efficient fine-tuning
- Prepare and format medical instruction datasets
- Train models with medical domain knowledge
- Evaluate model performance on clinical tasks
- Deploy fine-tuned models for inference

---
## Practice 1: Environment Setup and Installation

### 🎯 Learning Objectives
- Install essential libraries for medical LLM fine-tuning
- Set up GPU environment and verify resources
- Configure logging and monitoring tools

### 📖 Key Concepts
**Required Libraries:**
- `transformers`: Hugging Face model library
- `peft`: Parameter-Efficient Fine-Tuning
- `datasets`: Dataset loading and processing
- `bitsandbytes`: Quantization support (QLoRA)
- `accelerate`: Distributed training utilities

In [ ]:
# 1.1 Install required packages
# Note: Run this cell only once or in a fresh environment

!pip install -q transformers==4.36.0
!pip install -q peft==0.7.1
!pip install -q datasets==2.16.0
!pip install -q bitsandbytes==0.41.3
!pip install -q accelerate==0.25.0
!pip install -q scipy

print("✅ All packages installed successfully!")

In [ ]:
# 1.2 Import libraries and check GPU availability
import torch
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType
)
from datasets import Dataset, load_dataset
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Check GPU availability
print("=" * 60)
print("🖥️  System Information")
print("=" * 60)
print(f"PyTorch version: {torch.__version__}")
print(f"Transformers version: {transformers.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"Current GPU memory allocated: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")
else:
    print("⚠️  No GPU available. Training will be slow on CPU.")

print("\n✅ Environment setup complete!")

---
## Practice 2: Loading Pre-trained Models

### 🎯 Learning Objectives
- Load pre-trained language models from Hugging Face
- Understand model architecture and tokenization
- Prepare models for fine-tuning

### 📖 Key Concepts
**Model Selection:**
- Small models (7B): Faster training, less memory
- We'll use a small GPT-2 model for quick demonstration
- In production: Use LLaMA-7B, Mistral-7B, or similar

In [ ]:
# 2.1 Load base model and tokenizer
def load_base_model(model_name="gpt2"):
    """
    Load pre-trained model and tokenizer
    
    For production use:
    - model_name = "meta-llama/Llama-2-7b-hf" (requires access token)
    - model_name = "mistralai/Mistral-7B-v0.1"
    """
    print(f"Loading model: {model_name}")
    print("=" * 60)
    
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # Add padding token if not present
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        print("✅ Added pad_token = eos_token")
    
    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    )
    
    # Model information
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"\n📊 Model Statistics:")
    print(f"  Total parameters: {total_params:,}")
    print(f"  Trainable parameters: {trainable_params:,}")
    print(f"  Vocabulary size: {len(tokenizer)}")
    print(f"  Max sequence length: {tokenizer.model_max_length}")
    
    return model, tokenizer

# Load the model
base_model, tokenizer = load_base_model()
print("\n✅ Model loaded successfully!")

In [ ]:
# 2.2 Test base model inference
def test_base_model(model, tokenizer, prompt="Describe the symptoms of diabetes:"):
    """Test the base model before fine-tuning"""
    print("Testing base model...")
    print("=" * 60)
    print(f"Prompt: {prompt}")
    print("\nGenerated response:")
    
    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt")
    
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}
    
    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(response)
    print("\n" + "=" * 60)
    
    return response

# Test the model
base_response = test_base_model(base_model, tokenizer)

---
## Practice 3: Preparing Medical Datasets

### 🎯 Learning Objectives
- Create medical instruction datasets
- Format data for instruction tuning
- Implement data preprocessing pipelines

### 📖 Key Concepts
**Instruction Format:**
```
### Instruction:
{instruction}

### Input:
{input}

### Response:
{output}
```

In [ ]:
# 3.1 Create sample medical instruction dataset
def create_medical_dataset():
    """Create a sample medical instruction dataset"""
    
    # Sample medical QA pairs
    data = [
        {
            "instruction": "Explain the symptoms of the given condition.",
            "input": "Type 2 Diabetes",
            "output": "Type 2 diabetes symptoms include increased thirst, frequent urination, increased hunger, unintended weight loss, fatigue, blurred vision, slow-healing sores, and frequent infections. These symptoms develop slowly over time and may not be immediately noticeable."
        },
        {
            "instruction": "Provide treatment recommendations for the following condition.",
            "input": "Hypertension",
            "output": "Treatment for hypertension includes lifestyle modifications (reduced sodium intake, regular exercise, weight management, stress reduction) and medications such as ACE inhibitors, ARBs, diuretics, beta-blockers, or calcium channel blockers. Regular monitoring of blood pressure is essential."
        },
        {
            "instruction": "Describe the diagnostic criteria for the following disease.",
            "input": "Pneumonia",
            "output": "Pneumonia diagnosis involves clinical symptoms (fever, cough, difficulty breathing), physical examination (abnormal breath sounds), and diagnostic tests including chest X-ray showing infiltrates, complete blood count, and possibly sputum culture to identify the causative organism."
        },
        {
            "instruction": "Explain the mechanism of action for the medication.",
            "input": "Metformin",
            "output": "Metformin is a biguanide that works by decreasing hepatic glucose production, decreasing intestinal absorption of glucose, and improving insulin sensitivity by increasing peripheral glucose uptake and utilization. It does not cause hypoglycemia when used as monotherapy."
        },
        {
            "instruction": "List the risk factors for the given condition.",
            "input": "Coronary Artery Disease",
            "output": "Risk factors for coronary artery disease include hypertension, high cholesterol, smoking, diabetes, obesity, physical inactivity, family history, age (men >45, women >55), and chronic stress. Modifiable risk factors should be addressed through lifestyle changes and medical management."
        },
        {
            "instruction": "Explain the pathophysiology of the disease.",
            "input": "Asthma",
            "output": "Asthma is characterized by chronic airway inflammation leading to bronchial hyperresponsiveness and reversible airflow obstruction. Triggers cause mast cell degranulation, release of inflammatory mediators, smooth muscle contraction, mucus hypersecretion, and airway edema, resulting in wheezing, coughing, and difficulty breathing."
        },
        {
            "instruction": "Describe preventive measures for the condition.",
            "input": "Stroke",
            "output": "Stroke prevention includes controlling hypertension, managing diabetes, maintaining healthy cholesterol levels, not smoking, limiting alcohol intake, regular exercise, maintaining healthy weight, eating a balanced diet low in saturated fats, and taking prescribed anticoagulants or antiplatelet medications when indicated."
        },
        {
            "instruction": "Explain the differential diagnosis for the symptoms.",
            "input": "Chest pain",
            "output": "Differential diagnosis for chest pain includes cardiac causes (myocardial infarction, angina, pericarditis), pulmonary causes (pneumonia, pulmonary embolism, pneumothorax), gastrointestinal causes (GERD, esophageal spasm), musculoskeletal causes (costochondritis), and anxiety/panic attacks. Immediate evaluation is needed to rule out life-threatening conditions."
        }
    ]
    
    # Create DataFrame
    df = pd.DataFrame(data)
    
    print("📊 Medical Dataset Created")
    print("=" * 60)
    print(f"Total samples: {len(df)}")
    print(f"\nSample entry:")
    print(f"Instruction: {df.iloc[0]['instruction']}")
    print(f"Input: {df.iloc[0]['input']}")
    print(f"Output: {df.iloc[0]['output'][:100]}...")
    
    return df

# Create dataset
medical_df = create_medical_dataset()

In [ ]:
# 3.2 Format dataset for instruction tuning
def format_instruction(sample):
    """Format sample into instruction-following format"""
    instruction = sample['instruction']
    input_text = sample['input']
    output = sample['output']
    
    # Create formatted prompt
    if input_text:
        prompt = f"""### Instruction:
{instruction}

### Input:
{input_text}

### Response:
{output}"""
    else:
        prompt = f"""### Instruction:
{instruction}

### Response:
{output}"""
    
    return prompt

# Apply formatting
medical_df['text'] = medical_df.apply(format_instruction, axis=1)

print("Formatted sample:")
print("=" * 60)
print(medical_df['text'].iloc[0])
print("\n" + "=" * 60)

In [ ]:
# 3.3 Tokenize dataset
def tokenize_function(examples, tokenizer, max_length=512):
    """Tokenize the dataset"""
    return tokenizer(
        examples['text'],
        truncation=True,
        max_length=max_length,
        padding='max_length',
        return_tensors=None
    )

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(medical_df[['text']])

# Tokenize
tokenized_dataset = dataset.map(
    lambda x: tokenize_function(x, tokenizer),
    batched=True,
    remove_columns=['text']
)

print("✅ Dataset tokenized successfully!")
print(f"Number of samples: {len(tokenized_dataset)}")
print(f"Features: {tokenized_dataset.features}")

---
## Practice 4: Configuring LoRA for PEFT

### 🎯 Learning Objectives
- Understand LoRA configuration parameters
- Apply LoRA to the base model
- Verify trainable parameter reduction

### 📖 Key Concepts
**LoRA Configuration:**
- `r`: Rank (typically 8-16 for medical tasks)
- `lora_alpha`: Scaling factor (typically 2 × r)
- `target_modules`: Which layers to adapt (q_proj, v_proj)
- `lora_dropout`: Regularization (0.05-0.1)

**Memory Saving:** W = W₀ + BA where rank(B) = rank(A) = r

In [ ]:
# 4.1 Configure LoRA
def configure_lora(rank=8, alpha=16, dropout=0.05):
    """
    Configure LoRA for parameter-efficient fine-tuning
    
    Parameters:
    - rank (r): Low-rank dimension (4-64)
    - alpha: Scaling parameter (typically 2*r)
    - dropout: Dropout probability for LoRA layers
    """
    print("🔧 Configuring LoRA")
    print("=" * 60)
    
    lora_config = LoraConfig(
        r=rank,
        lora_alpha=alpha,
        target_modules=["c_attn"],  # For GPT-2; use ["q_proj", "v_proj"] for LLaMA
        lora_dropout=dropout,
        bias="none",
        task_type=TaskType.CAUSAL_LM
    )
    
    print(f"✅ LoRA Configuration:")
    print(f"  Rank (r): {rank}")
    print(f"  Alpha: {alpha}")
    print(f"  Dropout: {dropout}")
    print(f"  Target modules: {lora_config.target_modules}")
    
    return lora_config

# Create LoRA configuration
lora_config = configure_lora(rank=8, alpha=16, dropout=0.05)

In [ ]:
# 4.2 Apply LoRA to the model
def apply_lora(model, lora_config):
    """Apply LoRA configuration to the model"""
    print("\n🔄 Applying LoRA to model...")
    print("=" * 60)
    
    # Get original parameter count
    original_params = sum(p.numel() for p in model.parameters())
    original_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    # Apply LoRA
    model = get_peft_model(model, lora_config)
    
    # Get new parameter count
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    # Calculate percentages
    trainable_percent = 100 * trainable_params / total_params
    memory_reduction = 100 * (1 - trainable_params / original_params)
    
    print("📊 Parameter Statistics:")
    print(f"\n  Before LoRA:")
    print(f"    Total parameters: {original_params:,}")
    print(f"    Trainable parameters: {original_trainable:,}")
    print(f"\n  After LoRA:")
    print(f"    Total parameters: {total_params:,}")
    print(f"    Trainable parameters: {trainable_params:,}")
    print(f"    Trainable %: {trainable_percent:.2f}%")
    print(f"    Memory reduction: {memory_reduction:.2f}%")
    
    print("\n✅ LoRA applied successfully!")
    
    return model

# Apply LoRA
peft_model = apply_lora(base_model, lora_config)

# Print trainable parameters
print("\n" + "=" * 60)
print("Trainable Parameters:")
peft_model.print_trainable_parameters()

---
## Practice 5: Training with Medical Instructions

### 🎯 Learning Objectives
- Configure training arguments for medical LLM fine-tuning
- Implement training loop with proper monitoring
- Save and manage checkpoints

### 📖 Key Concepts
**Training Configuration:**
- Learning rate: 3e-4 (higher than full fine-tuning)
- Batch size: 4-8 (with gradient accumulation)
- Epochs: 3-5 for small datasets
- Warmup: 5-10% of total steps

In [ ]:
# 5.1 Configure training arguments
def setup_training_args(output_dir="./medical_llm_finetuned"):
    """Setup training arguments for fine-tuning"""
    
    training_args = TrainingArguments(
        output_dir=output_dir,
        
        # Training hyperparameters
        num_train_epochs=3,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        learning_rate=3e-4,
        
        # Optimization
        warmup_steps=10,
        weight_decay=0.01,
        
        # Logging and saving
        logging_steps=10,
        save_steps=50,
        save_total_limit=2,
        
        # Mixed precision
        fp16=torch.cuda.is_available(),
        
        # Other settings
        remove_unused_columns=False,
        report_to="none",  # Disable wandb/tensorboard for this demo
    )
    
    print("⚙️  Training Configuration")
    print("=" * 60)
    print(f"Output directory: {output_dir}")
    print(f"Number of epochs: {training_args.num_train_epochs}")
    print(f"Batch size per device: {training_args.per_device_train_batch_size}")
    print(f"Gradient accumulation steps: {training_args.gradient_accumulation_steps}")
    print(f"Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
    print(f"Learning rate: {training_args.learning_rate}")
    print(f"Warmup steps: {training_args.warmup_steps}")
    print(f"Mixed precision (fp16): {training_args.fp16}")
    
    return training_args

# Setup training arguments
training_args = setup_training_args()

In [ ]:
# 5.2 Create data collator and trainer
def setup_trainer(model, tokenizer, train_dataset, training_args):
    """Setup the Trainer for fine-tuning"""
    
    # Data collator for language modeling
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False  # Causal language modeling (not masked)
    )
    
    # Create trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        data_collator=data_collator,
    )
    
    print("✅ Trainer initialized successfully!")
    print(f"Training samples: {len(train_dataset)}")
    
    return trainer

# Setup trainer
trainer = setup_trainer(peft_model, tokenizer, tokenized_dataset, training_args)

In [ ]:
# 5.3 Train the model
print("🚀 Starting training...")
print("=" * 60)
print("This may take a few minutes depending on your hardware.")
print("")

# Train
train_result = trainer.train()

# Print training results
print("\n" + "=" * 60)
print("📊 Training Complete!")
print("=" * 60)
print(f"Training loss: {train_result.training_loss:.4f}")
print(f"Training time: {train_result.metrics['train_runtime']:.2f} seconds")
print(f"Samples per second: {train_result.metrics['train_samples_per_second']:.2f}")

# Save the model
trainer.save_model()
print("\n✅ Model saved successfully!")

---
## Practice 6: Model Evaluation and Testing

### 🎯 Learning Objectives
- Test the fine-tuned model on medical queries
- Compare responses before and after fine-tuning
- Evaluate model performance qualitatively

### 📖 Key Concepts
**Evaluation Metrics:**
- Qualitative: Response relevance and accuracy
- Quantitative: Perplexity, BLEU score
- Medical-specific: Clinical accuracy, safety

In [ ]:
# 6.1 Generate responses with fine-tuned model
def generate_response(model, tokenizer, prompt, max_length=150):
    """Generate response using the fine-tuned model"""
    
    # Format prompt
    formatted_prompt = f"""### Instruction:
{prompt}

### Response:
"""
    
    # Tokenize
    inputs = tokenizer(formatted_prompt, return_tensors="pt")
    
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}
    
    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract only the response part
    if "### Response:" in response:
        response = response.split("### Response:")[1].strip()
    
    return response

# Test prompts
test_prompts = [
    "Explain the symptoms of Type 2 Diabetes.",
    "What are the treatment options for hypertension?",
    "Describe the mechanism of action of aspirin."
]

print("🧪 Testing Fine-Tuned Model")
print("=" * 60)

for i, prompt in enumerate(test_prompts, 1):
    print(f"\nTest {i}:")
    print(f"Prompt: {prompt}")
    print("\nResponse:")
    response = generate_response(peft_model, tokenizer, prompt)
    print(response)
    print("\n" + "-" * 60)

In [ ]:
# 6.2 Compare base model vs fine-tuned model
def compare_models(base_model, finetuned_model, tokenizer, prompt):
    """Compare responses from base and fine-tuned models"""
    
    print("🔍 Model Comparison")
    print("=" * 60)
    print(f"Prompt: {prompt}")
    print("\n" + "=" * 60)
    
    # Base model response
    print("\n📘 Base Model Response:")
    base_response = test_base_model(base_model, tokenizer, prompt)
    
    # Fine-tuned model response
    print("\n📗 Fine-Tuned Model Response:")
    ft_response = generate_response(finetuned_model, tokenizer, prompt)
    print(ft_response)
    
    print("\n" + "=" * 60)
    print("Observation: The fine-tuned model should provide more")
    print("structured and medically accurate responses.")
    
    return base_response, ft_response

# Compare on a medical query
comparison_prompt = "What are the symptoms of pneumonia?"
base_resp, ft_resp = compare_models(base_model, peft_model, tokenizer, comparison_prompt)

---
## Practice 7: Inference and Deployment

### 🎯 Learning Objectives
- Load saved models for inference
- Merge LoRA weights with base model
- Prepare model for production deployment

### 📖 Key Concepts
**Deployment Options:**
1. Keep LoRA adapters separate (smaller storage)
2. Merge adapters into base model (faster inference)
3. Quantize for edge deployment

In [ ]:
# 7.1 Load the fine-tuned model
def load_finetuned_model(model_path="./medical_llm_finetuned"):
    """Load the fine-tuned model from checkpoint"""
    
    print("📂 Loading fine-tuned model...")
    print("=" * 60)
    
    from peft import PeftModel
    
    # Load base model
    base = AutoModelForCausalLM.from_pretrained(
        "gpt2",
        device_map="auto",
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    )
    
    # Load PEFT model
    model = PeftModel.from_pretrained(base, model_path)
    
    print(f"✅ Model loaded from: {model_path}")
    
    return model

# Load the model
loaded_model = load_finetuned_model()
print("\n✅ Model ready for inference!")

In [ ]:
# 7.2 Merge LoRA weights (optional)
def merge_lora_weights(peft_model):
    """Merge LoRA adapters into the base model"""
    
    print("🔄 Merging LoRA weights...")
    print("=" * 60)
    
    # Merge and unload
    merged_model = peft_model.merge_and_unload()
    
    print("✅ LoRA weights merged successfully!")
    print("   The model is now a standard model without adapters.")
    print("   This is useful for:")
    print("   - Faster inference (no adapter overhead)")
    print("   - Easier deployment")
    print("   - Compatibility with standard inference pipelines")
    
    return merged_model

# Merge weights
merged_model = merge_lora_weights(loaded_model)

In [ ]:
# 7.3 Save merged model for deployment
def save_for_deployment(model, tokenizer, output_path="./medical_llm_deployed"):
    """Save the final model for deployment"""
    
    print("💾 Saving model for deployment...")
    print("=" * 60)
    
    # Save model
    model.save_pretrained(output_path)
    tokenizer.save_pretrained(output_path)
    
    print(f"✅ Model saved to: {output_path}")
    print("\nDeployment files:")
    print("  - pytorch_model.bin (model weights)")
    print("  - config.json (model configuration)")
    print("  - tokenizer files")
    
    return output_path

# Save for deployment
deployment_path = save_for_deployment(merged_model, tokenizer)

---
## Practice 8: Monitoring and Optimization

### 🎯 Learning Objectives
- Monitor training metrics and model performance
- Identify common issues and solutions
- Optimize hyperparameters for better results

### 📖 Key Concepts
**Common Issues:**
- Overfitting: Model memorizes training data
- Catastrophic forgetting: Loss of general knowledge
- Poor convergence: Learning rate too high/low

In [ ]:
# 8.1 Best practices summary
def print_best_practices():
    """Print best practices for medical LLM fine-tuning"""
    
    print("📋 Best Practices for Medical LLM Fine-Tuning")
    print("=" * 60)
    
    practices = {
        "✅ DO's": [
            "Start with small LoRA rank (r=8) and increase if needed",
            "Use higher learning rates (3e-4 to 1e-3) for PEFT",
            "Monitor both training loss and validation performance",
            "Save checkpoints regularly",
            "Test on diverse medical queries",
            "Use mixed precision training (FP16) for efficiency",
            "Validate with medical experts before deployment"
        ],
        "❌ DON'Ts": [
            "Don't overtrain - stop when validation loss plateaus",
            "Don't use too large batch sizes (causes memory issues)",
            "Don't skip data quality checks",
            "Don't deploy without safety testing",
            "Don't ignore catastrophic forgetting",
            "Don't use medical models for final diagnosis"
        ]
    }
    
    for category, items in practices.items():
        print(f"\n{category}:")
        for item in items:
            print(f"  • {item}")
    
    print("\n" + "=" * 60)
    print("⚠️  Important Reminder:")
    print("Medical LLMs are assistive tools and should not replace")
    print("professional medical judgment. Always require expert validation.")

print_best_practices()

In [ ]:
# 8.2 Hyperparameter recommendations
def print_hyperparameter_guide():
    """Print hyperparameter tuning guide"""
    
    print("\n⚙️  Hyperparameter Tuning Guide")
    print("=" * 60)
    
    guide = {
        "LoRA Rank (r)": {
            "r=4": "Simple tasks, very limited data",
            "r=8": "Most medical NLP tasks (recommended start)",
            "r=16": "Complex medical reasoning",
            "r=32-64": "Multi-task learning, large datasets"
        },
        "Learning Rate": {
            "1e-3 to 5e-3": "PEFT methods (LoRA, Adapters)",
            "1e-5 to 5e-5": "Full fine-tuning",
            "Higher": "Small datasets, quick adaptation",
            "Lower": "Large datasets, careful tuning"
        },
        "Batch Size": {
            "4-8": "Most scenarios with gradient accumulation",
            "16-32": "Large GPU memory available",
            "1-2": "Very large models (70B+)"
        },
        "Epochs": {
            "3-5": "Small datasets (< 1K samples)",
            "2-3": "Medium datasets (1K-10K samples)",
            "1-2": "Large datasets (> 10K samples)"
        }
    }
    
    for param, values in guide.items():
        print(f"\n📊 {param}:")
        for setting, description in values.items():
            print(f"  • {setting}: {description}")
    
    print("\n" + "=" * 60)
    print("💡 Tip: Start with conservative settings and adjust based on")
    print("   validation performance. Monitor for overfitting!")

print_hyperparameter_guide()

---
## 🎯 Practice Complete!

### 🎓 Summary of What We Learned:

1. **Environment Setup**: Installing libraries and checking GPU resources
2. **Model Loading**: Loading pre-trained models from Hugging Face
3. **Data Preparation**: Creating and formatting medical instruction datasets
4. **LoRA Configuration**: Applying parameter-efficient fine-tuning
5. **Training**: Fine-tuning with medical instructions
6. **Evaluation**: Testing and comparing model responses
7. **Deployment**: Saving and preparing models for production
8. **Best Practices**: Guidelines for successful medical LLM fine-tuning

### 🔑 Key Takeaways:

- **LoRA reduces trainable parameters by 99%+** while maintaining performance
- **Medical instruction tuning** improves domain-specific responses
- **Proper dataset formatting** is crucial for effective fine-tuning
- **Safety and validation** are paramount in medical applications

### 📈 Performance Gains:

| Metric | Base Model | Fine-Tuned Model | Improvement |
|--------|------------|------------------|-------------|
| Medical Accuracy | Low | High | ⬆️ ⬆️ ⬆️ |
| Response Structure | Poor | Good | ⬆️ ⬆️ |
| Domain Knowledge | Generic | Specialized | ⬆️ ⬆️ ⬆️ |
| Memory Usage | - | 99% less | ⬇️ ⬇️ ⬇️ |

### 🚀 Next Steps:

1. **Experiment with larger models**: Try LLaMA-7B, Mistral-7B
2. **Use real medical datasets**: MedInstruct, HealthCareMagic, MedQA
3. **Implement QLoRA**: 4-bit quantization for even better efficiency
4. **Add evaluation metrics**: BLEU, ROUGE, clinical accuracy
5. **Deploy with safety checks**: Implement guardrails and monitoring

### 📚 Additional Resources:

- [Hugging Face PEFT Documentation](https://huggingface.co/docs/peft)
- [LoRA Paper](https://arxiv.org/abs/2106.09685)
- [QLoRA Paper](https://arxiv.org/abs/2305.14314)
- [Medical Instruction Datasets](https://github.com/Kent0n-Li/ChatDoctor)

### ⚠️ Important Reminders:

1. **Never deploy medical models without expert validation**
2. **Always include appropriate disclaimers**
3. **Comply with healthcare regulations (HIPAA, GDPR, etc.)**
4. **Monitor for bias and ensure fairness**
5. **Keep models updated with latest medical knowledge**

---

## 🙏 Thank You!

**Questions?**
- Email: homin.park@ghent.ac.kr | powersimmani@gmail.com
- GitHub: [Your Repository]
- Community: Join Medical AI forums

**Good luck with your medical LLM fine-tuning journey! 🏥🤖**